In [ ]:
import numpy as np
import torch


# Model  $f(\cdot)=Relu(tanh((\cdot) + bias))$

a model for implementig BP-TRW and BP for experiments of figures 6, 7, and 8

In [ ]:
dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# bias = False : Initial bias vectors with zero vectors
# bias = True : Initial bias vectors with N(initial_mean , initial_var )
# N_layer_list : A list which contains number of neurons in each layer. For example N_layer_list = [100,20,30]  creates a network with input layer of size 100, a single hidden layer of size 20, and output layer of size 30
# Forward and backward weight matrices are initialized from N(initial_mean,initial_var )

class my_network1():

    def __init__(self ,  N_layer_list , bias = False , initial_mean = 0 ,  initial_var = 0.1):
        self.norm_W_list = []
        self.norm_B_list = []
        self.bias_vec = []
        self.N_layer_list = N_layer_list

        self.W =[]
        self.L =[]
        self.Z =[]
        self.delta_BP =[]
        self.delta_FA =[]

        self.E_BP =[]
        self.E_FA =[]


        self.B =[]

        self.DeltaW_bp = []
        self.DeltaW_fa = []

        self.Delta_bias_bp = []
        self.Delta_bias_fa = []

        self.bias_vec.append(None)
        self.Delta_bias_bp.append(None)
        self.Delta_bias_fa.append(None)



        self.num_layers  = len(self.N_layer_list)-1  
        for i in range( self.num_layers ):

          self.W.append( ((torch.randn(  N_layer_list[i]  , N_layer_list[i+1] )) ) .to(device).to(dtype)  * initial_var + initial_mean )


          self.DeltaW_bp.append( torch.zeros_like(self.W[-1]).to(device) )
          self.DeltaW_fa.append( torch.zeros_like(self.W[-1]).to(device) )
          
          self.B.append( ((torch.randn(self.N_layer_list[i] , self.N_layer_list[i+1]))  ).t().to(device).to(dtype)   * initial_var  + initial_mean   ) 

          if bias==True :
             self.bias_vec.append (  torch.randn( 1  , self.N_layer_list[i+1] ) .to(device).to(dtype)   * initial_var  + initial_mean )
          elif bias==False :
             self.bias_vec.append (  torch.zeros( 1  , self.N_layer_list[i+1] ) .to(device).to(dtype)    )

          self.Delta_bias_bp.append( torch.zeros( 1  , self.N_layer_list[i+1] ).to(device).to(dtype)  )
          self.Delta_bias_fa.append( torch.zeros( 1  , self.N_layer_list[i+1] ) .to(device).to(dtype)  )


          
        for i in range( self.num_layers + 1 ):
          self.L.append( None )
          self.Z.append( None )
          self.delta_FA.append( None  )
          self.delta_BP.append( None  )

          self.E_FA.append( None )
          self.E_BP.append( None )


    def forward( self , X ):
        self.L[0] = X
        for i in range( self.num_layers  ):
          self.Z[i+1] = torch.matmul( self.L[i] , self.W[i] ) +  self.bias_vec [i+1]
          self.L[i+1] = self.activation1( self.Z[i+1] ) 
        return self.L[-1]
    








    def backprop(self, E_l ):


      

        self.E_BP[-1] = E_l
        self.delta_BP[-1] =  torch.mul( self.derivative_activation1( self.Z[-1] ) ,  self.E_BP[-1]  )
      

        for i in range(len(self.N_layer_list)-2):
     
          self.E_BP[ -2 - i] = torch.matmul(   self.delta_BP[-1-i] , torch.transpose( self.W[-1-i] , 0,1)   )
          self.delta_BP[-2-i] =  torch.mul (  self.derivative_activation1(self.Z[-2-i] ) , self.E_BP[ -2 - i] )



        for i in range(len(self.N_layer_list)-1):
          self.DeltaW_bp[i] = torch.matmul(   torch.transpose( self.L[i]  , 0,1) , self.delta_BP[i+1]    )  
           
        for i in range(1,len(self.N_layer_list)):
          self.Delta_bias_bp[i] = self.delta_BP[i] . sum(dim=0) .view(1,-1)

        return self.DeltaW_bp , self.Delta_bias_bp



    def BP_TRW(self,  E ):


  
        self.E_FA[-1 ] = E



        self.delta_FA[-1] =  torch.mul( self.derivative_activation1( self.Z[-1] ) ,  self.E_FA[-1 ]   )
 

        for i in range(len(self.N_layer_list)-2):
          self.E_FA[ -2 - i] = torch.matmul(   self.delta_FA[-1-i]   ,  self.B[-1-i]    )
       
          self.delta_FA[-2-i] =  torch.mul (self.derivative_activation1(self.Z[-2-i] ) , self.E_FA[ -2 - i ]     )
      
    


        for i in range(len(self.N_layer_list)-1):

          self.DeltaW_fa[i] = torch.matmul(   torch.transpose( self.L[i]  , 0,1) , self.delta_FA[i+1]    )  
           
        for i in range(1,len(self.N_layer_list)):
          self.Delta_bias_fa[i] = self.delta_FA[i] . sum(dim=0) .view(1,-1)

        return self.DeltaW_fa , self.Delta_bias_fa



    def activation1(self,x):
 

      return torch.tanh(torch.relu(x))




    def derivative_activation1(self,x):



      m = (x > 0) * 1

      x = torch.cosh(x)
      x = torch.mul ( x , x )
      x = 1/x

      return torch.mul ( x , m )
      


    def set_learning_rate(self , lr):
      self.lr = lr



    def step_W(self,deltaW):
      for i in range(len(self.N_layer_list)-1):
        self.W[i] = torch.add( self.W[i] , deltaW[i] ,alpha= self.lr )

    

    def step_bias(self,delta_bias):
      for i in range(1,len(self.N_layer_list)):
        self.bias_vec[i] = torch.add( self.bias_vec[i] , delta_bias[i] ,alpha= self.lr )

   



    def normalize_W(self , norm = None ):
      if norm==None:
        for i in range(len(self.N_layer_list)-1):
          self.W[i] = self.W[i] / self.W[i].norm() * self.norm_W_list[i]
      elif type(norm)==list :   
        for i in range(len(self.N_layer_list)-1):
          self.W[i] = self.W[i] / self.W[i].norm() * norm  [i]
      else : 
        for i in range(len(self.N_layer_list)-1):
          self.W[i] = self.W[i] / self.W[i].norm() * norm


    def normalize_B(self , norm = None ):
      if norm==None:
        for i in range(len(self.N_layer_list)-1):
          self.B[i] = self.B[i] / self.B[i].norm() * self.norm_B_list[i]
      elif type(norm)==list :   
        for i in range(len(self.N_layer_list)-1):
          self.B[i] = self.B[i] / self.B[i].norm() * norm  [i]          
      else : 
        for i in range(len(self.N_layer_list)-1):
          self.B[i] = self.B[i] / self.B[i].norm() * norm


    def match_B_norm_to_W_norm(self):
      for i in range(len(self.N_layer_list)-1):
        self.B[i] = self.B[i] / self.B[i].norm() *self.W[i].norm() 


    def save_norm( self ):
      self.norm_W_list = []
      self.norm_B_list = []
      for i in range(len(self.N_layer_list)-1):

        self.norm_W_list . append( self.W[i] .norm() )
        self.norm_B_list . append( self.B[i] .norm() )


    
    def column_normalize_W(self , norm = None ):
        if norm==None:
          for l in range(len(self.N_layer_list)-1):
            self.W[l] = torch.mul( torch.div( self.W[l] ,  self.W[l].norm(dim=0).view( [1,-1] )  )  , self.norm_W_list[l].view( [1,-1] ) )
        elif type(norm)==list : 
          for l in range(len(self.N_layer_list)-1):
            self.W[l] = torch.div( self.W[l] ,  self.W[l].norm(dim=0).view( [1,-1] )  )  * norm[l]
        else : 
          for l in range(len(self.N_layer_list)-1):
            self.W[l] = torch.div( self.W[l] ,  self.W[l].norm(dim=0).view( [1,-1] )  )  * norm

    def column_normalize_B(self , norm = None ):
        if norm==None:
          for l in range(len(self.N_layer_list)-1):
            self.B[l] = torch.mul( torch.div( self.B[l] ,  self.B[l].norm(dim=0).view( [1,-1] )  ) , self.norm_B_list[l].view( [1,-1] ) )

        elif type(norm)==list : 
          for l in range(len(self.N_layer_list)-1):
            self.B[l] = torch.div( self.B[l] ,  self.B[l].norm(dim=0).view( [1,-1] )  )  * norm[l]
        else : 
          for l in range(len(self.N_layer_list)-1):
            self.B[l] = torch.div( self.B[l] ,  self.B[l].norm(dim=0).view( [1,-1] )  )  * norm
  


    def seed_norms( self , from1 , till1 ):
      for l in range(len(self.N_layer_list)-1):
        self.norm_W_list .append( torch.rand(  self.N_layer_list[l+1]   ) .to(device) .to(dtype) *( till1 - from1 ) + from1 )
        self.norm_B_list .append( torch.rand(  self.N_layer_list[l]   ) .to(device).to(dtype) *( till1 - from1 ) + from1 )

    def set_norm( self , norm ):
      self.norm_W_list = []
      self.norm_B_list = []
      for i in range(len(self.N_layer_list)-1):
        self.norm_W_list . append(norm )
        self.norm_B_list . append(norm )


    def set_norm_LIST( self , norm_LIST ):
      self.norm_W_list = []
      for i in range(len(self.N_layer_list)-1):
        self.norm_W_list . append(norm_LIST[i] )
